In [6]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import contractions

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danh\AppData\Roaming\nltk_data...


True

In [4]:
df = pd.read_csv('Dataset\IMDB-Dataset.csv')
df = df.drop_duplicates()

In [7]:
stopWords = set(stopwords.words('english'))

In [12]:
def preprocess_text(text):
    wl = WordNetLemmatizer()

    soup = BeautifulSoup(text, 'html.parser') # remove html tag

    text = soup.get_text()
    text = contractions.fix(text) # expand contractions (viết tắt)

    emoji_clean = re.compile("["
                             u'\U0001F600-\U0001F64F' # emoticons
                             u'\U0001F300-\U0001F5FF' # symbols & pictographs
                             u'\U0001F680-\U0001F6FF' # transport & map symbols
                             u'\U0001F1E0-\U0001F1FF' # flags (iOS)
                             u'\U00002702-\U000027B0'
                             u'\U000024C2-\U0001F251'
                             "]+", flags=re.UNICODE)
    
    text = emoji_clean.sub(r'', text)
    text = re.sub(r'\.(?=\S)', '. ', text) # add space after a sentence
    text = re.sub(r'http\S+', '', text) # remove url
    text = ''.join(
        [word.lower() for word in text if word not in string.punctuation]
    ) # lower text and remove punctuation
    text = ' '.join(
        [wl.lemmatize(word) for word in text.split() if word not in stopWords and word.isalpha()]
    )
    return text


In [13]:
df['review_processed'] = df['review'].apply(preprocess_text)

C:\Users\Danh\AppData\Local\Temp\ipykernel_19172\2489398871.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser') # remove html tag


In [14]:
df[:3]

,review,sentiment,review_processed
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...


In [15]:
df.to_csv('Dataset\IMDB-Dataset.csv')